# Demo for the Bayesian Iterative Quantum Amplitude Estimation (BIQAE) algorithm


In [1]:

import sys
sys.path.insert(0, '../src')

from biae import BayesianIQAE as BIQAE

import numpy as np

from qiskit.circuit import QuantumCircuit
from qiskit_algorithms import EstimationProblem
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2

Define the estimation problem.

In [2]:
theta_in_degrees = 45
a = np.sin(theta_in_degrees / 180*np.pi) ** 2   # amplitude to be estimated

class BernoulliA(QuantumCircuit):
    """A circuit representing the Bernoulli A operator."""

    def __init__(self, probability):
        super().__init__(1)  # circuit on 1 qubit

        theta_p = 2 * np.arcsin(np.sqrt(probability))
        self.ry(theta_p, 0)


class BernoulliQ(QuantumCircuit):
    """A circuit representing the Bernoulli Q operator."""

    def __init__(self, probability):
        super().__init__(1)  # circuit on 1 qubit

        self._theta_p = 2 * np.arcsin(np.sqrt(probability))
        self.ry(2 * self._theta_p, 0)

    def power(self, k):
        # implement the efficient power of Q
        q_k = QuantumCircuit(1)
        q_k.ry(2 * k * self._theta_p, 0)
        return q_k


A = BernoulliA(a)
Q = BernoulliQ(a)

problem = EstimationProblem(
    state_preparation=A,  # A operator
    grover_operator=Q,  # mock Grover operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

Create a sampler instance.

In [3]:
simulator_backend = AerSimulator()
sampler = SamplerV2(simulator_backend)

Create a BIQAE instance.

In [4]:
biqae = BIQAE(
    epsilon_target=1e-4,  # target accuracy
    alpha=0.05, # confidence level
    sampler=sampler,
    min_ratio = 2, 
    confint_method="beta"
)

Run the BIQAE algorithm.

In [5]:
biqae_result = biqae.estimate(problem, bayes=True, show_details=True)



                BAYESIAN ITERATIVE QUANTUM AMPLITUDE ESTIMATION                 
Target epsilon: 0.000100
Confidence level: 0.950000
Confidence interval method: beta
Bayesian mode: Enabled
--------------------------------------------------------------------------------
Maximum number of stages: 13
Number of shots taken in each iteration: 10
--------------------------------------------------------------------------------
Stage: 0    Iteration: 0
Current theta interval: [0, 0.25]
Found k=0, running time: 0.0000 seconds
Circuit constructed with 0 Q operators. Depth: 2. Construction time: 0.0010 seconds
Sampled 10 shots, running time: 0.02 seconds
Accumulated quantum sample complexity: 0
--------------------------------------------------------------------------------
Stage: 0    Iteration: 1
Current theta interval: [0.08617397152134404, 0.21745327003750006]
Found k=0, running time: 0.0000 seconds
Circuit constructed with 0 Q operators. Depth: 2. Construction time: 0.0000 seconds
Sampled 1

Retrieve the results

In [6]:
ci = biqae_result.confidence_interval
N_oracle = biqae_result.num_oracle_queries
print(f"The interval estimate for the amplitude is: {ci}")
print(f"The number of oracle queries is: {N_oracle}")
if ci[0] <= a and ci[1] >= a and (ci[1] - ci[0]) < 2 * biqae.epsilon_target:
    print("Successfully estimated the amplitude with the desired accuracy.")
else:
    print("Failed to estimate the amplitude with the desired accuracy.")


The interval estimate for the amplitude is: [0.499881726790913, 0.5000812945231454]
The number of oracle queries is: 57650
Successfully estimated the amplitude with the desired accuracy.
